In [ ]:
import numpy as np
import os
import sys
import cv2

path = r'C:\Users\ThreeThree\Desktop\compiledcodes\datasci\Regala'

def read_images(path=path, sz=None):
    X, y = [], []
    for subdir, dirs, files in os.walk(path):
        for file in files:
            try:
                if file == ".directory":
                    continue
                filepath = os.path.join(subdir, file)
                im = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)

                if im is None:
                    print("Error: Unable to read image:", filepath)
                    continue

                print("Image shape before resize:", im.shape)

                if sz is not None:
                    im = cv2.resize(im, sz)

                if im is not None:
                    X.append(np.asarray(im, dtype=np.uint8))
                    y.append(0)  
                    
            except IOError as e:
                print(f"I/O Error({e.errno}): {e.strerror}")
            except:
                print("Unexpected error:", sys.exc_info()[0])
                raise

    return [X, y]

# Test the function
[X, y] = read_images(sz=(200, 200))
print(f"Loaded {len(X)} images.")


In [4]:
def face_rec():
    names = ['Regala', 'not_Regala']
    [X, y] = read_images(path)
    y = np.asarray(y, dtype=np.int32)

    model = cv2.face.EigenFaceRecognizer_create()
    model.train(X, y)

    camera = cv2.VideoCapture(0)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    while True:
        ret, img = camera.read()
        if not ret:
            break

        faces = face_cascade.detectMultiScale(img, 1.3, 5)

        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
            gray = cv2.cvtColor(img[y:y + h, x:x + w], cv2.COLOR_BGR2GRAY)
            roi = cv2.resize(gray, (185, 216), interpolation=cv2.INTER_LINEAR)

            try:
                params = model.predict(roi)
                label = names[params[0]]
                cv2.putText(img, label + ", " + str(params[1]), (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            except:
                continue

        cv2.imshow("camera", img)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    camera.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    face_rec()

Image shape before resize: (480, 640)
Image shape before resize: (480, 640)
Image shape before resize: (480, 640)
Image shape before resize: (480, 640)
Image shape before resize: (480, 640)
Image shape before resize: (480, 640)
Image shape before resize: (480, 640)
Image shape before resize: (480, 640)
Image shape before resize: (480, 640)
Image shape before resize: (480, 640)
Image shape before resize: (480, 640)
Image shape before resize: (480, 640)
Image shape before resize: (480, 640)
Image shape before resize: (480, 640)
Image shape before resize: (480, 640)
Image shape before resize: (480, 640)
Image shape before resize: (480, 640)
Image shape before resize: (480, 640)
Image shape before resize: (480, 640)
Image shape before resize: (480, 640)
Image shape before resize: (480, 640)
Image shape before resize: (480, 640)
Image shape before resize: (480, 640)
Image shape before resize: (480, 640)
Image shape before resize: (480, 640)
Image shape before resize: (480, 640)
Image shape 

In [29]:
import cv2
from deepface import DeepFace

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def detect_faces(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return faces


def recognize_emotion(image):
    detected_faces = detect_faces(image)
    for (x, y, w, h) in detected_faces:
        face = image[y:y+h, x:x+w]
        results = DeepFace.analyze(face, actions=['emotion'], enforce_detection=False)
        for result in results:
            emotion = max(result['emotion'].items(), key=lambda x: x[1])[0]
            cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(image, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
    return image

def main():
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_with_emotion = recognize_emotion(frame)
        cv2.imshow('Emotion Detection', frame_with_emotion)

        # Break loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break


    cap.release()
    cv2.destroyAllWindows()

if __name__=='__main__':
    main()